<h1>2b. Machine Learning using tf.estimator </h1>

In this notebook, we will create a machine learning model using tf.estimator and evaluate its performance.  The dataset is rather small (7700 samples), so we can do it all in-memory.  We will also simply pass the raw data in as-is. 

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil

print(tf.__version__)

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.8.0


Read data created in the previous chapter.

In [2]:
# In CSV, label is the first column, after the features, followed by the key
CSV_COLUMNS = ['fare_amount', 'pickuplon','pickuplat','dropofflon','dropofflat','passengers', 'key']
FEATURES = CSV_COLUMNS[1:len(CSV_COLUMNS) - 1]
LABEL = CSV_COLUMNS[0]

df_train = pd.read_csv('./taxi-train.csv', header = None, names = CSV_COLUMNS)
df_valid = pd.read_csv('./taxi-valid.csv', header = None, names = CSV_COLUMNS)
df_test = pd.read_csv('./taxi-test.csv', header = None, names = CSV_COLUMNS)

In [3]:
print(df_train.head())

   fare_amount  pickuplon  pickuplat  dropofflon  dropofflat  passengers  key
0         12.0 -73.987625  40.750617  -73.971163   40.785180           1    0
1          4.5 -73.963620  40.774363  -73.953485   40.772665           1    1
2          4.5 -73.989649  40.756633  -73.985597   40.765662           1    2
3         10.0 -73.993950  40.727524  -74.006584   40.744240           1    3
4          2.5 -73.950223  40.668960  -73.948112   40.668872           6    4


In [4]:
print(FEATURES)

['pickuplon', 'pickuplat', 'dropofflon', 'dropofflat', 'passengers']


<h2> Train and eval input functions to read from Pandas Dataframe </h2>

In [22]:
# TODO: Create an appropriate input_fn to read the training data
def make_train_input_fn(df, num_epochs=10):
  return tf.estimator.inputs.pandas_input_fn(
    #ADD CODE HERE
    x = df,
    y = df['fare_amount'],
    batch_size = 128,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000
  )

In [37]:
# TODO: Create an appropriate input_fn to read the validation data
def make_eval_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    #ADD CODE HERE
    x = df,
    y = df['fare_amount'],
    batch_size = 64,
    num_epochs = 1,
    shuffle = True
    #queue_capacity = 1000
  )

Our input function for predictions is the same except we don't provide a label

In [23]:
# TODO: Create an appropriate prediction_input_fn
def make_prediction_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    #ADD CODE HERE
    x = df,
    batch_size = 128,
    num_epochs = 1,
    shuffle = False,
    queue_capacity = 1000
  )

### Create feature columns for estimator

In [9]:
# TODO: Create feature columns
feat_columns = [
  tf.feature_column.numeric_column(x) for x in FEATURES
]

In [10]:
print(feat_columns)

[_NumericColumn(key='pickuplon', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='pickuplat', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='dropofflon', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='dropofflat', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='passengers', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


<h3> Linear Regression with tf.Estimator framework </h3>

In [24]:
tf.logging.set_verbosity(tf.logging.INFO)

OUTDIR = 'taxi_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

# TODO: Train a linear regression model
model = tf.estimator.LinearRegressor(feat_columns)#ADD CODE HERE

model.train(make_train_input_fn(df_train))#ADD CODE HERE

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp83jxen21', '_tf_random_seed': None, '_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3296ac8e80>, '_task_type': 'worker', '_save_summary_steps': 100, '_session_config': None, '_num_worker_replicas': 1, '_is_chief': True, '_task_id': 0, '_save_checkpoints_secs': 600, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_num_ps_replicas': 0, '_evaluation_master': '', '_service': None, '_global_id_in_cluster': 0, '_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_train_distribute': None}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp83jxen21/model.ckpt.
INFO:tensorflow:loss = 30077

Evaluate on the validation data (we should defer using the test data to after we have selected a final model).

In [26]:
def print_rmse(model, df):
  metrics = model.evaluate(input_fn = make_eval_input_fn(df))
  #print('RMSE on dataset = {}'.format(name, np.sqrt(metrics['average_loss'])))
  print('RMSE on dataset = {}'.format(np.sqrt(metrics['average_loss'])))
print_rmse(model, df_valid)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-08-01:13:17
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp83jxen21/model.ckpt-608
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-08-01:13:17
INFO:tensorflow:Saving dict for global step 608: average_loss = 109.13058, global_step = 608, loss = 6729.7188
RMSE on dataset = 10.446557998657227


This is nowhere near our benchmark (RMSE of $6 or so on this data), but it serves to demonstrate what TensorFlow code looks like.  Let's use this model for prediction.

In [31]:
# TODO: Predict from the estimator model we trained using test dataset
predictions = model.predict(make_prediction_input_fn(df_test))
for pred in predictions:
  print(pred)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp83jxen21/model.ckpt-608
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([11.072613], dtype=float32)}
{'predictions': array([11.070025], dtype=float32)}
{'predictions': array([11.070901], dtype=float32)}
{'predictions': array([11.068412], dtype=float32)}
{'predictions': array([11.072347], dtype=float32)}
{'predictions': array([11.072124], dtype=float32)}
{'predictions': array([11.070683], dtype=float32)}
{'predictions': array([11.070714], dtype=float32)}
{'predictions': array([11.072505], dtype=float32)}
{'predictions': array([11.070321], dtype=float32)}
{'predictions': array([11.072614], dtype=float32)}
{'predictions': array([11.072782], dtype=float32)}
{'predictions': array([11.065955], dtype=float32)}
{'predictions': array([11.069934], dtype=float32)}
{'predictions': arr

{'predictions': array([11.188174], dtype=float32)}
{'predictions': array([11.068762], dtype=float32)}
{'predictions': array([11.0722685], dtype=float32)}
{'predictions': array([11.07146], dtype=float32)}
{'predictions': array([11.130351], dtype=float32)}
{'predictions': array([11.187938], dtype=float32)}
{'predictions': array([11.070893], dtype=float32)}
{'predictions': array([11.247995], dtype=float32)}
{'predictions': array([11.12782], dtype=float32)}
{'predictions': array([11.071118], dtype=float32)}
{'predictions': array([11.067981], dtype=float32)}
{'predictions': array([11.056749], dtype=float32)}
{'predictions': array([11.069767], dtype=float32)}
{'predictions': array([11.130533], dtype=float32)}
{'predictions': array([11.0711565], dtype=float32)}
{'predictions': array([11.303713], dtype=float32)}
{'predictions': array([11.071388], dtype=float32)}
{'predictions': array([11.072236], dtype=float32)}
{'predictions': array([11.072913], dtype=float32)}
{'predictions': array([11.07046

{'predictions': array([11.070312], dtype=float32)}
{'predictions': array([11.070378], dtype=float32)}
{'predictions': array([11.3043], dtype=float32)}
{'predictions': array([11.363751], dtype=float32)}
{'predictions': array([11.070193], dtype=float32)}
{'predictions': array([11.131336], dtype=float32)}
{'predictions': array([11.0707035], dtype=float32)}
{'predictions': array([11.1251955], dtype=float32)}
{'predictions': array([11.070454], dtype=float32)}
{'predictions': array([11.07129], dtype=float32)}
{'predictions': array([11.071229], dtype=float32)}
{'predictions': array([11.129418], dtype=float32)}
{'predictions': array([11.307774], dtype=float32)}
{'predictions': array([11.070791], dtype=float32)}
{'predictions': array([11.069794], dtype=float32)}
{'predictions': array([11.187805], dtype=float32)}
{'predictions': array([11.071037], dtype=float32)}
{'predictions': array([11.070754], dtype=float32)}
{'predictions': array([11.305774], dtype=float32)}
{'predictions': array([11.06955]

{'predictions': array([11.128205], dtype=float32)}
{'predictions': array([11.303735], dtype=float32)}
{'predictions': array([11.070195], dtype=float32)}
{'predictions': array([11.070534], dtype=float32)}
{'predictions': array([11.071228], dtype=float32)}
{'predictions': array([11.245559], dtype=float32)}
{'predictions': array([11.128923], dtype=float32)}
{'predictions': array([11.073207], dtype=float32)}
{'predictions': array([11.125024], dtype=float32)}
{'predictions': array([11.070685], dtype=float32)}
{'predictions': array([11.07111], dtype=float32)}
{'predictions': array([11.301514], dtype=float32)}
{'predictions': array([11.304512], dtype=float32)}
{'predictions': array([11.070763], dtype=float32)}
{'predictions': array([11.070246], dtype=float32)}
{'predictions': array([11.364319], dtype=float32)}
{'predictions': array([11.304162], dtype=float32)}
{'predictions': array([11.072513], dtype=float32)}
{'predictions': array([11.0705805], dtype=float32)}
{'predictions': array([11.13021

This explains why the RMSE was so high -- the model essentially predicts the same amount for every trip.  Would a more complex model help? Let's try using a deep neural network.  The code to do this is quite straightforward as well.

<h3> Deep Neural Network regression </h3>

In [34]:
# TODO: Copy your LinearRegressor estimator and replace with DNNRegressor. Remember to add a list of hidden units i.e. [32, 8, 2]
tf.logging.set_verbosity(tf.logging.INFO)

OUTDIR = 'taxi_trained_DNN'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

# TODO: Train a DNN model
model = tf.estimator.DNNRegressor([32,8,2],feat_columns)#ADD CODE HERE

model.train(make_train_input_fn(df_train))#ADD CODE HERE

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpyjct3cc4', '_tf_random_seed': None, '_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f32740f5fd0>, '_task_type': 'worker', '_save_summary_steps': 100, '_session_config': None, '_num_worker_replicas': 1, '_is_chief': True, '_task_id': 0, '_save_checkpoints_secs': 600, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_num_ps_replicas': 0, '_evaluation_master': '', '_service': None, '_global_id_in_cluster': 0, '_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_train_distribute': None}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpyjct3cc4/model.ckpt.
INFO:tensorflow:loss = 32178

We are not beating our benchmark with either model ... what's up?  Well, we may be using TensorFlow for Machine Learning, but we are not yet using it well.  That's what the rest of this course is about!

But, for the record, let's say we had to choose between the two models. We'd choose the one with the lower validation error. Finally, we'd measure the RMSE on the test data with this chosen model.

<h2> Benchmark dataset </h2>

Let's do this on the benchmark dataset.

In [38]:
import google.datalab.bigquery as bq
import numpy as np
import pandas as pd

def create_query(phase, EVERY_N):
  """
  phase: 1 = train 2 = valid
  """
  base_query = """
SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  EXTRACT(DAYOFWEEK FROM pickup_datetime) * 1.0 AS dayofweek,
  EXTRACT(HOUR FROM pickup_datetime) * 1.0 AS hourofday,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count * 1.0 AS passengers,
  CONCAT(CAST(pickup_datetime AS STRING), CAST(pickup_longitude AS STRING), CAST(pickup_latitude AS STRING), CAST(dropoff_latitude AS STRING), CAST(dropoff_longitude AS STRING)) AS key
FROM
  `nyc-tlc.yellow.trips`
WHERE
  trip_distance > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  AND passenger_count > 0
  """

  if EVERY_N == None:
    if phase < 2:
      # Training
      query = "{0} AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 4) < 2".format(base_query)
    else:
      # Validation
      query = "{0} AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 4) = {1}".format(base_query, phase)
  else:
    query = "{0} AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), {1}) = {2}".format(base_query, EVERY_N, phase)
    
  return query

query = create_query(2, 100000)
df = bq.Query(query).execute().result().to_dataframe()

In [39]:
print_rmse(model, df)

ValueError: Index for x and y are mismatched.
Index for x: RangeIndex(start=0, stop=1665, step=1)
Index for y: RangeIndex(start=0, stop=10927, step=1)


In [40]:
print(df)

       fare_amount  dayofweek  hourofday  pickuplon  pickuplat  dropofflon  \
0            10.00        2.0        0.0 -73.954365  40.770057  -73.975082   
1            20.83        2.0        0.0 -73.942335  40.788055  -73.912777   
2             8.00        1.0        0.0 -73.991715  40.722245  -74.009032   
3            13.00        1.0        0.0 -73.986675  40.751657  -73.996910   
4             5.50        7.0        0.0 -73.974162  40.748116  -73.981144   
5             7.50        1.0        0.0 -73.973170  40.752902  -73.991065   
6             8.50        4.0        0.0 -73.986589  40.760160  -73.965994   
7             4.50        4.0        0.0 -73.968046  40.759780  -73.958755   
8             7.00        2.0        0.0 -73.943575  40.835945  -73.940480   
9            16.00        1.0        0.0 -74.010172  40.707770  -73.978355   
10           49.57        2.0        0.0 -73.782009  40.647253  -73.926416   
11            9.50        1.0        0.0 -73.964678  40.806717  

RMSE on benchmark dataset is <b>9.41</b> (your results will vary because of random seeds).

This is not only way more than our original benchmark of 6.00, but it doesn't even beat our distance-based rule's RMSE of 8.02.

Fear not -- you have learned how to write a TensorFlow model, but not to do all the things that you will have to do to your ML model performant. We will do this in the next chapters. In this chapter though, we will get our TensorFlow model ready for these improvements.

In a software sense, the rest of the labs in this chapter will be about refactoring the code so that we can improve it.

## Challenge Exercise

Create a neural network that is capable of finding the volume of a cylinder given the radius of its base (r) and its height (h). Assume that the radius and height of the cylinder are both in the range 0.5 to 2.0. Simulate the necessary training dataset.
<p>
Hint (highlight to see):
<p style='color:white'>
The input features will be r and h and the label will be $\pi r^2 h$
Create random values for r and h and compute V.
Your dataset will consist of r, h and V.
Then, use a DNN regressor.
Make sure to generate enough data.
</p>

Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License